In [1]:
import numpy as np
import pandas as pd

In [2]:
# declare a list tasks whose products you want to use as inputs
upstream = None
product = None
min_rows_categories = None # minimo de objetos que devem conter essas categorias para ser desconsiderado
df_input_path = '../../dados/X_trainToronto.csv'

In [3]:
# Parameters
min_rows_categories = 50
df_input_path = "../../dados/X_trainToronto.csv"
product = {
    "data": "/home/guilherme/\xc1rea de Trabalho/Estudo Faculdade/Disciplinas/minera\xe7\xe3o de dados/Desafio/pipelines/processamento_features/output/processar_categories_data.parquet",
    "nb": "/home/guilherme/\xc1rea de Trabalho/Estudo Faculdade/Disciplinas/minera\xe7\xe3o de dados/Desafio/pipelines/processamento_features/output/output_processar_categories.ipynb",
}


In [4]:
df_reg = pd.read_csv(df_input_path)
df_reg = df_reg[['business_id', 'categories']]
df_reg

,business_id,categories
0,vHzWmPWHN4J1hRR3W3AMQg,"Wine Bars, Tapas/Small Plates, Restaurants, Ba..."
1,15to24Q-otAHmto7FzsWRg,"Beauty & Spas, Hair Salons, Barbers, Shopping,..."
2,8aqKdf4G4AAir8k_Kdslvg,"Health & Medical, Medical Centers"
3,uxU1vr5AhhkTQ83X0bpeyg,"Health & Medical, Hospitals, Medical Centers"
4,f702hTJoqdR34Jn23C7d1A,"Automotive, Bikes, Shopping, Auto Repair, Spor..."
...,...,...
17577,Zp5xil8pC6Bm-fFOWIbh0Q,"Restaurants, Halal, Mexican"
17578,o2811xZ0ufRi7WcsqWiGzw,"Food, Coffee & Tea"
17579,v9T9m_TbpLuaC2Kud5UbLA,"Sewing & Alterations, Shopping, Local Services"
17580,bzGnAIe9OiBHGWJE-dZ_8g,"Chicken Wings, Bars, Nightlife, Restaurants, P..."


## Transformar categories em dummies

In [5]:
df_reg.categories = df_reg.categories.fillna('')
df_reg.categories = df_reg.categories.astype(str)
df_reg.categories = df_reg.categories.str.split(',')
df_reg.categories = df_reg.categories.apply(lambda x: [s.strip().lower() for s in x])
df_reg.categories

0        [wine bars, tapas/small plates, restaurants, b...
1        [beauty & spas, hair salons, barbers, shopping...
2                      [health & medical, medical centers]
3           [health & medical, hospitals, medical centers]
4        [automotive, bikes, shopping, auto repair, spo...
                               ...                        
17577                        [restaurants, halal, mexican]
17578                                 [food, coffee & tea]
17579     [sewing & alterations, shopping, local services]
17580    [chicken wings, bars, nightlife, restaurants, ...
17581    [orthotics, massage therapy, chiropractors, he...
Name: categories, Length: 17582, dtype: object

In [6]:
from collections import defaultdict

all_categories = defaultdict(int)
for c_list in df_reg.categories:
    if type(c_list) is float:
        c_list = []
    for c in c_list:
        all_categories[c.strip().lower()] = all_categories[c.strip().lower()] + 1

best_categories = pd.Series(all_categories)
best_categories = best_categories[best_categories > min_rows_categories]
others_categories = set(all_categories.keys()).difference(set(best_categories.index))
best_categories

wine bars                 127
tapas/small plates         94
restaurants              7418
bars                     1443
nightlife                1616
                         ... 
tires                      52
chicken shop               65
dog walkers                51
banks & credit unions      52
music venues              146
Length: 203, dtype: int64

In [7]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
df_cat_dummies = pd.DataFrame(mlb.fit_transform(df_reg.categories),columns=mlb.classes_, index=df_reg.index)

if min_rows_categories > 0:
    df_cat_dummies['category_others_categories'] = df_cat_dummies.apply(lambda x: x[list(others_categories)].sum(), axis=1)
    df_cat_dummies = df_cat_dummies.drop(list(others_categories), axis=1)


df_reg = pd.concat([df_reg, df_cat_dummies], axis=1)
df_reg = df_reg.rename(columns={cat:'category_' + cat for cat in best_categories.index})
df_reg

,business_id,categories,category_accessories,category_active life,category_acupuncture,category_american (new),category_american (traditional),category_art galleries,category_arts & crafts,category_arts & entertainment,...,category_venues & event spaces,category_veterinarians,category_vietnamese,category_vintage & consignment,category_waxing,category_wine & spirits,category_wine bars,category_women's clothing,category_yoga,category_others_categories
0,vHzWmPWHN4J1hRR3W3AMQg,"[wine bars, tapas/small plates, restaurants, b...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,15to24Q-otAHmto7FzsWRg,"[beauty & spas, hair salons, barbers, shopping...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,8aqKdf4G4AAir8k_Kdslvg,"[health & medical, medical centers]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,uxU1vr5AhhkTQ83X0bpeyg,"[health & medical, hospitals, medical centers]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,f702hTJoqdR34Jn23C7d1A,"[automotive, bikes, shopping, auto repair, spo...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17577,Zp5xil8pC6Bm-fFOWIbh0Q,"[restaurants, halal, mexican]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17578,o2811xZ0ufRi7WcsqWiGzw,"[food, coffee & tea]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17579,v9T9m_TbpLuaC2Kud5UbLA,"[sewing & alterations, shopping, local services]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
17580,bzGnAIe9OiBHGWJE-dZ_8g,"[chicken wings, bars, nightlife, restaurants, ...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
category_columns = [x for x in df_reg.columns if x.startswith('category_')]
df_reg[category_columns] = df_reg[category_columns].apply(lambda x: x.astype(float))

In [9]:
df_reg.to_parquet(product['data'])

## Teste manual

In [10]:
from IPython.display import display
with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(df_reg.iloc[13]['categories'])
    display(df_reg.iloc[[13]][category_columns])
    display('----------------------------------')
    display(df_reg.iloc[542]['categories'])
    display(df_reg.iloc[[542]][category_columns])
    display('----------------------------------')
    display(df_reg.iloc[5389]['categories'])
    display(df_reg.iloc[[5389]][category_columns])

['sandwiches', 'american (traditional)', 'breakfast & brunch', 'restaurants']

,category_accessories,category_active life,category_acupuncture,category_american (new),category_american (traditional),category_art galleries,category_arts & crafts,category_arts & entertainment,category_asian fusion,category_auto repair,category_automotive,category_bagels,category_bakeries,category_banks & credit unions,category_barbeque,category_barbers,category_bars,category_beauty & spas,category_beer,category_bikes,category_blow dry/out services,category_books,category_bookstores,category_breakfast & brunch,category_breweries,category_bridal,category_bubble tea,category_burgers,category_cafes,category_canadian (new),category_car dealers,category_car rental,category_cards & stationery,category_caribbean,category_caterers,category_cheese shops,category_chicken shop,category_chicken wings,category_chinese,category_chiropractors,category_chocolatiers & shops,category_cinema,category_cocktail bars,category_coffee & tea,category_comfort food,category_computers,category_contractors,category_convenience stores,category_cosmetic dentists,category_cosmetics & beauty supply,category_dance clubs,category_day spas,category_delis,category_dentists,category_department stores,category_desserts,category_dim sum,category_diners,category_dive bars,category_doctors,category_dog walkers,category_donuts,category_drugstores,category_dry cleaning,category_dry cleaning & laundry,category_education,category_electronics,category_ethnic food,category_event planning & services,category_eyelash service,category_eyewear & opticians,category_fashion,category_fast food,category_financial services,category_fish & chips,category_fitness & instruction,category_florists,category_flowers & gifts,category_food,category_food court,category_food delivery services,category_food trucks,category_french,category_furniture stores,category_gastropubs,category_general dentistry,category_gift shops,category_gluten-free,category_greek,category_grocery,category_gyms,category_hair removal,category_hair salons,category_halal,category_health & medical,category_health markets,category_hobby shops,category_home & garden,category_home cleaning,category_home decor,category_home services,category_hotels,category_hotels & travel,category_ice cream & frozen yogurt,category_imported food,category_indian,category_it services & computer repair,category_italian,category_japanese,category_jewelry,category_juice bars & smoothies,category_kitchen & bath,category_korean,category_laser hair removal,category_latin american,category_laundry services,category_libraries,category_local flavor,category_local services,category_lounges,category_mags,category_makeup artists,category_massage,category_massage therapy,category_meat shops,category_medical centers,category_medical spas,category_mediterranean,category_men's clothing,category_mexican,category_middle eastern,category_mobile phone repair,category_mobile phones,category_modern european,category_movers,category_music & dvds,category_music & video,category_music venues,category_nail salons,category_naturopathic/holistic,category_nightlife,category_noodles,category_optometrists,category_pakistani,category_parks,category_party & event planning,category_performing arts,category_pet groomers,category_pet services,category_pet sitting,category_pet stores,category_pets,category_photography stores & services,category_physical therapy,category_pilates,category_pizza,category_portuguese,category_printing services,category_professional services,category_public services & government,category_public transportation,category_pubs,category_ramen,category_real estate,category_restaurants,category_salad,category_sandwiches,category_seafood,category_sewing & alterations,category_shoe stores,category_shopping,category_skin care,category_soup,category_specialty food,category_specialty schools,category_sporting goods,category_sports bars,category_sports clubs,category_sports wear,category_steakhouses,category_sushi bars,category_tanning,category_tapas bars,categor

'----------------------------------'

['food', 'cheese shops', 'specialty food']

,category_accessories,category_active life,category_acupuncture,category_american (new),category_american (traditional),category_art galleries,category_arts & crafts,category_arts & entertainment,category_asian fusion,category_auto repair,category_automotive,category_bagels,category_bakeries,category_banks & credit unions,category_barbeque,category_barbers,category_bars,category_beauty & spas,category_beer,category_bikes,category_blow dry/out services,category_books,category_bookstores,category_breakfast & brunch,category_breweries,category_bridal,category_bubble tea,category_burgers,category_cafes,category_canadian (new),category_car dealers,category_car rental,category_cards & stationery,category_caribbean,category_caterers,category_cheese shops,category_chicken shop,category_chicken wings,category_chinese,category_chiropractors,category_chocolatiers & shops,category_cinema,category_cocktail bars,category_coffee & tea,category_comfort food,category_computers,category_contractors,category_convenience stores,category_cosmetic dentists,category_cosmetics & beauty supply,category_dance clubs,category_day spas,category_delis,category_dentists,category_department stores,category_desserts,category_dim sum,category_diners,category_dive bars,category_doctors,category_dog walkers,category_donuts,category_drugstores,category_dry cleaning,category_dry cleaning & laundry,category_education,category_electronics,category_ethnic food,category_event planning & services,category_eyelash service,category_eyewear & opticians,category_fashion,category_fast food,category_financial services,category_fish & chips,category_fitness & instruction,category_florists,category_flowers & gifts,category_food,category_food court,category_food delivery services,category_food trucks,category_french,category_furniture stores,category_gastropubs,category_general dentistry,category_gift shops,category_gluten-free,category_greek,category_grocery,category_gyms,category_hair removal,category_hair salons,category_halal,category_health & medical,category_health markets,category_hobby shops,category_home & garden,category_home cleaning,category_home decor,category_home services,category_hotels,category_hotels & travel,category_ice cream & frozen yogurt,category_imported food,category_indian,category_it services & computer repair,category_italian,category_japanese,category_jewelry,category_juice bars & smoothies,category_kitchen & bath,category_korean,category_laser hair removal,category_latin american,category_laundry services,category_libraries,category_local flavor,category_local services,category_lounges,category_mags,category_makeup artists,category_massage,category_massage therapy,category_meat shops,category_medical centers,category_medical spas,category_mediterranean,category_men's clothing,category_mexican,category_middle eastern,category_mobile phone repair,category_mobile phones,category_modern european,category_movers,category_music & dvds,category_music & video,category_music venues,category_nail salons,category_naturopathic/holistic,category_nightlife,category_noodles,category_optometrists,category_pakistani,category_parks,category_party & event planning,category_performing arts,category_pet groomers,category_pet services,category_pet sitting,category_pet stores,category_pets,category_photography stores & services,category_physical therapy,category_pilates,category_pizza,category_portuguese,category_printing services,category_professional services,category_public services & government,category_public transportation,category_pubs,category_ramen,category_real estate,category_restaurants,category_salad,category_sandwiches,category_seafood,category_sewing & alterations,category_shoe stores,category_shopping,category_skin care,category_soup,category_specialty food,category_specialty schools,category_sporting goods,category_sports bars,category_sports clubs,category_sports wear,category_steakhouses,category_sushi bars,category_tanning,category_tapas bars,categor

'----------------------------------'

['organic stores', 'health markets', 'specialty food', 'grocery', 'food']

,category_accessories,category_active life,category_acupuncture,category_american (new),category_american (traditional),category_art galleries,category_arts & crafts,category_arts & entertainment,category_asian fusion,category_auto repair,category_automotive,category_bagels,category_bakeries,category_banks & credit unions,category_barbeque,category_barbers,category_bars,category_beauty & spas,category_beer,category_bikes,category_blow dry/out services,category_books,category_bookstores,category_breakfast & brunch,category_breweries,category_bridal,category_bubble tea,category_burgers,category_cafes,category_canadian (new),category_car dealers,category_car rental,category_cards & stationery,category_caribbean,category_caterers,category_cheese shops,category_chicken shop,category_chicken wings,category_chinese,category_chiropractors,category_chocolatiers & shops,category_cinema,category_cocktail bars,category_coffee & tea,category_comfort food,category_computers,category_contractors,category_convenience stores,category_cosmetic dentists,category_cosmetics & beauty supply,category_dance clubs,category_day spas,category_delis,category_dentists,category_department stores,category_desserts,category_dim sum,category_diners,category_dive bars,category_doctors,category_dog walkers,category_donuts,category_drugstores,category_dry cleaning,category_dry cleaning & laundry,category_education,category_electronics,category_ethnic food,category_event planning & services,category_eyelash service,category_eyewear & opticians,category_fashion,category_fast food,category_financial services,category_fish & chips,category_fitness & instruction,category_florists,category_flowers & gifts,category_food,category_food court,category_food delivery services,category_food trucks,category_french,category_furniture stores,category_gastropubs,category_general dentistry,category_gift shops,category_gluten-free,category_greek,category_grocery,category_gyms,category_hair removal,category_hair salons,category_halal,category_health & medical,category_health markets,category_hobby shops,category_home & garden,category_home cleaning,category_home decor,category_home services,category_hotels,category_hotels & travel,category_ice cream & frozen yogurt,category_imported food,category_indian,category_it services & computer repair,category_italian,category_japanese,category_jewelry,category_juice bars & smoothies,category_kitchen & bath,category_korean,category_laser hair removal,category_latin american,category_laundry services,category_libraries,category_local flavor,category_local services,category_lounges,category_mags,category_makeup artists,category_massage,category_massage therapy,category_meat shops,category_medical centers,category_medical spas,category_mediterranean,category_men's clothing,category_mexican,category_middle eastern,category_mobile phone repair,category_mobile phones,category_modern european,category_movers,category_music & dvds,category_music & video,category_music venues,category_nail salons,category_naturopathic/holistic,category_nightlife,category_noodles,category_optometrists,category_pakistani,category_parks,category_party & event planning,category_performing arts,category_pet groomers,category_pet services,category_pet sitting,category_pet stores,category_pets,category_photography stores & services,category_physical therapy,category_pilates,category_pizza,category_portuguese,category_printing services,category_professional services,category_public services & government,category_public transportation,category_pubs,category_ramen,category_real estate,category_restaurants,category_salad,category_sandwiches,category_seafood,category_sewing & alterations,category_shoe stores,category_shopping,category_skin care,category_soup,category_specialty food,category_specialty schools,category_sporting goods,category_sports bars,category_sports clubs,category_sports wear,category_steakhouses,category_sushi bars,category_tanning,category_tapas bars,categor